In [ ]:
# 📦 Instalar bibliotecas interativas
!pip install plotly dash jupyter-dash pandas openpyxl


In [ ]:
# 📚 Importações
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output


In [ ]:
# 📁 Carregamento da base
arquivo = '/content/Manifestacoes_Abril2025.xlsx'
df = pd.read_excel(arquivo)

# Ajustar colunas caso necessário
if df.columns[0] != 'COD_MANIFESTACAO':
    df.columns = df.iloc[0]
    df = df[1:]
    df.columns.name = None

# Renomear colunas
df = df.rename(columns={
    "COD_MANIFESTACAO": "COD_MANIFESTACAO",
    "MOTIVO": "MOTIVO",
    "DATA_MANIF": "DATA_MANIF",
    "DIA_SEM": "DIA_SEM",
    "DATA_OCOR": "DATA_OCOR",
    "HORA_OCOR": "HORA_OCOR",
    "LOCAL_REF": "LOCAL_REF",
    "DESCRIÇÃO": "DESCRICAO",
    "CLASSIFICACAO_MANIF": "CLASSIFICACAO_MANIF",
    df.columns[-1]: "LINHA"
})

# Converter tipos
df["DATA_MANIF"] = pd.to_datetime(df["DATA_MANIF"], errors='coerce')
df["HORA_OCOR"] = pd.to_numeric(df["HORA_OCOR"], errors='coerce')


In [ ]:
# 🚀 Criação do app Dash por classificação de manifestação
def iniciar_dashboard_por_classificacao(classe):
    dados = df[df["CLASSIFICACAO_MANIF"].str.upper() == classe.upper()].copy()

    app = JupyterDash(__name__)
    app.layout = html.Div([
        html.H1(f"Dashboard Interativo - {classe.title()}", style={"textAlign": "center"}),

        dcc.Dropdown(
            id='linha-dropdown',
            options=[{"label": linha, "value": linha} for linha in sorted(dados["LINHA"].dropna().unique())],
            placeholder="Selecione uma linha (opcional)",
            multi=True
        ),

        dcc.DatePickerRange(
            id='data-range',
            min_date_allowed=dados["DATA_MANIF"].min(),
            max_date_allowed=dados["DATA_MANIF"].max(),
            start_date=dados["DATA_MANIF"].min(),
            end_date=dados["DATA_MANIF"].max()
        ),

        dcc.Graph(id='grafico-motivos'),
        dcc.Graph(id='grafico-horas'),
        dcc.Graph(id='grafico-local')
    ])

    @app.callback(
        Output('grafico-motivos', 'figure'),
        Output('grafico-horas', 'figure'),
        Output('grafico-local', 'figure'),
        Input('linha-dropdown', 'value'),
        Input('data-range', 'start_date'),
        Input('data-range', 'end_date')
    )
    def atualizar_graficos(linhas_selecionadas, data_inicio, data_fim):
        dff = dados.copy()
        if linhas_selecionadas:
            dff = dff[dff["LINHA"].isin(linhas_selecionadas)]
        dff = dff[(dff["DATA_MANIF"] >= data_inicio) & (dff["DATA_MANIF"] <= data_fim)]

        fig1 = px.bar(dff["MOTIVO"].value_counts().nlargest(10).reset_index(),
                      x="index", y="MOTIVO", title="Top 10 Motivos")
        fig1.update_layout(xaxis_title="Motivo", yaxis_title="Frequência")

        fig2 = px.histogram(dff, x="HORA_OCOR", nbins=24, title="Distribuição por Hora")
        fig2.update_layout(xaxis_title="Hora do Dia", yaxis_title="Quantidade")

        fig3 = px.bar(dff["LOCAL_REF"].value_counts().nlargest(10).reset_index(),
                      x="index", y="LOCAL_REF", title="Top 10 Locais")
        fig3.update_layout(xaxis_title="Local", yaxis_title="Quantidade")

        return fig1, fig2, fig3

    app.run_server(mode='inline')


In [ ]:
# ✅ Executar dashboards interativos por classificação
# Use as chamadas abaixo individualmente no Colab para abrir o dashboard desejado:

# iniciar_dashboard_por_classificacao("RECLAMAÇÃO")
# iniciar_dashboard_por_classificacao("ELOGIO")
# iniciar_dashboard_por_classificacao("SUGESTÃO")
